In [ ]:
# 3-1-1 Service Requests (2025) — Data Cleaning & KPI Engineering
This notebook cleans the dataset and prepares a Power BI-ready file with KPI fields.

In [ ]:
##1)Imports

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
os.listdir()

In [ ]:
## 2) Load Data
Read the raw CSV (semicolon-separated).

In [ ]:
df = pd.read_csv(
    "3-1-1-service-requests.csv",
    sep=";",
    engine="python",
    on_bad_lines="skip"
)

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
## Data Quality Snapshot
Quick check of missing values for key fields used in KPIs and dashboard filters.

In [ ]:
dq = df[["Service request close date", "Local area", "Latitude", "Longitude"]].isna().mean().mul(100).round(2)
dq

In [ ]:
## 3) Parse Datetimes
- Open/modified timestamps include timezone → parse as UTC
- Close date has no timezone → keep it naive

In [ ]:
df["Service request open timestamp"] = pd.to_datetime(
    df["Service request open timestamp"],
    errors="coerce",
    utc=True
)

df["Last modified timestamp"] = pd.to_datetime(
    df["Last modified timestamp"],
    errors="coerce",
    utc=True
)

df["Service request close date"] = pd.to_datetime(
    df["Service request close date"],
    errors="coerce"
)

In [ ]:
df.info()

In [ ]:
## 4) KPI Engineering
We compute:
- `resolution_time_days`: days between open timestamp and close date
- `is_closed`: close date exists (True/False)

**Negative durations** can happen due to date inconsistencies (e.g., timezone vs date-only close field). For reporting, negative values are capped at **0**.

In [ ]:
open_ts = df["Service request open timestamp"].dt.tz_convert(None)
df["resolution_time_days"] = (df["Service request close date"] - open_ts).dt.days

In [ ]:
df["resolution_time_days"].describe()

In [ ]:
negative_count = (df["resolution_time_days"] < 0).sum()
negative_pct = negative_count / df["resolution_time_days"].notna().sum() * 100
negative_count, round(negative_pct, 2)

In [ ]:
df.loc[df["resolution_time_days"] < 0, "resolution_time_days"] = 0

In [ ]:
# Power BI-friendly nullable integer (keeps missing values)
df["resolution_time_days"] = df["resolution_time_days"].astype("Int64")
df["resolution_time_days"].dtype

In [ ]:
df["is_closed"] = df["Service request close date"].notna()
df["is_closed"].value_counts()

In [ ]:
backlog_pct = ((~df["is_closed"]).sum() / len(df)) * 100
round(backlog_pct, 2)

In [ ]:
## KPI Summary (2025)
Headline numbers used in the Power BI dashboard.

In [ ]:
kpi_summary = {
    "Total requests": int(len(df)),
    "Closed requests": int(df["is_closed"].sum()),
    "Open requests": int((~df["is_closed"]).sum()),
    "Backlog %": round(((~df["is_closed"]).sum() / len(df)) * 100, 2),
    "Median resolution (days) - closed only": int(df.loc[df["is_closed"], "resolution_time_days"].median()),
}
kpi_summary

In [ ]:
## Department KPI (Closed Requests Only)
We compute department-level median/mean resolution time. We also filter to departments with **>= 100** closed requests to avoid noisy results.

In [ ]:
dept_kpi = (
    df[df["is_closed"]]
    .groupby("Department")["resolution_time_days"]
    .agg(["count", "median", "mean"])
    .sort_values("median", ascending=False)
)
dept_kpi.head(10)

In [ ]:
dept_kpi_filtered = dept_kpi[dept_kpi["count"] >= 100]
dept_kpi_filtered.head(10)

In [ ]:
## 5) Export for Power BI
Select only the columns needed for the dashboard and export to CSV.

In [ ]:
powerbi_cols = [
    "Department",
    "Service request type",
    "Status",
    "Channel",
    "Local area",
    "Service request open timestamp",
    "Service request close date",
    "resolution_time_days",
    "is_closed",
]

df_powerbi = df[powerbi_cols]
df_powerbi.info()

In [ ]:
df_powerbi.to_csv("311_cleaned_2025_powerbi.csv", index=False)
"Saved: 311_cleaned_2025_powerbi.csv"